In [1]:
import sys
sys.path.append("../")

import pandas as pd
import plotly.graph_objects as go
from dateutil import parser
import datetime as dt

from api.oanda_api import OandaApi
from infrastructure.instrument_collection import instrumentCollection as ic
from db.db import DataDB

In [2]:
database = DataDB()

In [3]:
ic.LoadInstrumentsDB()

In [28]:
pairs = []
for pair, val in ic.instruments_dict.items():
    if val.ins_type == "CURRENCY" and "USD" in pair:
        pairs.append(pair)

In [29]:
pairs

['USD_CHF',
 'USD_THB',
 'USD_JPY',
 'NZD_USD',
 'USD_SGD',
 'USD_CAD',
 'AUD_USD',
 'USD_TRY',
 'USD_NOK',
 'USD_CNH',
 'USD_PLN',
 'USD_ZAR',
 'USD_CZK',
 'USD_DKK',
 'USD_SEK',
 'GBP_USD',
 'USD_MXN',
 'USD_HUF',
 'USD_HKD',
 'EUR_USD']

In [30]:
data = []
api = OandaApi()
for p in pairs:
    print(p)
    d_temp = api.get_candles_df(pair_name=p, granularity="D", count=400)
    d_temp['gain'] = ((d_temp['mid-c'] - d_temp['mid-o']) / d_temp['mid-c']) * 100
    d_temp['pair'] = p
    if '_USD' in p:
        d_temp['gain'] = d_temp['gain'] * -1
    data.append(d_temp)
candles_df = pd.concat(data)
candles_df.reset_index(drop=True, inplace=True)
candles_df.time = candles_df.time.dt.date

USD_CHF
USD_THB
USD_JPY
NZD_USD
USD_SGD
USD_CAD
AUD_USD
USD_TRY
USD_NOK
USD_CNH
USD_PLN
USD_ZAR
USD_CZK
USD_DKK
USD_SEK
GBP_USD
USD_MXN
USD_HUF
USD_HKD
EUR_USD


In [31]:
candles_df.tail()

,volume,time,mid-o,mid-h,mid-l,mid-c,bid-o,bid-h,bid-l,bid-c,ask-o,ask-h,ask-l,ask-c,gain,pair
7975,57279,2024-01-14,1.09474,1.09676,1.09336,1.09503,1.09463,1.09668,1.09328,1.09495,1.09486,1.09683,1.09343,1.09511,-0.026483,EUR_USD
7976,112047,2024-01-15,1.09488,1.09512,1.08621,1.08756,1.09471,1.09505,1.08613,1.08747,1.09506,1.09529,1.08628,1.08764,0.673066,EUR_USD
7977,100485,2024-01-16,1.08738,1.08846,1.08446,1.08821,1.08704,1.08839,1.08439,1.08813,1.08771,1.08854,1.08454,1.08829,-0.076272,EUR_USD
7978,102050,2024-01-17,1.08824,1.09068,1.08468,1.08763,1.08813,1.09060,1.08460,1.08754,1.08835,1.09075,1.08475,1.08772,0.056085,EUR_USD
7979,85492,2024-01-18,1.08746,1.08978,1.08657,1.08976,1.08729,1.08970,1.08649,1.08966,1.08763,1.08986,1.08665,1.08986,-0.211056,EUR_USD


In [32]:
calendar_data = database.query_all(DataDB.CALENDAR_COLL)

In [33]:
calendar_data_df = pd.DataFrame.from_dict(calendar_data)

In [34]:
calendar_data_df.head()

,date,country,category,event,symbol,actual,previous,forecast
0,2024-01-23,australia,business confidence,nab business confidence,NABSCONF,,-9,-7
1,2024-01-23,japan,interest rate,boj interest rate decision,BOJDTR,,-0.1%,-0.1%
2,2024-01-23,japan,balance of trade,balance of trade,JNTBAL,,¥-776.9B,¥ -200B
3,2024-01-24,germany,manufacturing pmi,hcob manufacturing pmi flash,GERMANYMANPMI,,43.3,44.0
4,2024-01-24,canada,interest rate,boc interest rate decision,CCLR,,5%,5%


In [35]:
calendar_data_df.date = calendar_data_df.date.dt.date

In [36]:
calendar_data_df

,date,country,category,event,symbol,actual,previous,forecast
0,2024-01-23,australia,business confidence,nab business confidence,NABSCONF,,-9,-7
1,2024-01-23,japan,interest rate,boj interest rate decision,BOJDTR,,-0.1%,-0.1%
2,2024-01-23,japan,balance of trade,balance of trade,JNTBAL,,¥-776.9B,¥ -200B
3,2024-01-24,germany,manufacturing pmi,hcob manufacturing pmi flash,GERMANYMANPMI,,43.3,44.0
4,2024-01-24,canada,interest rate,boc interest rate decision,CCLR,,5%,5%
...,...,...,...,...,...,...,...,...
611,2024-01-30,france,gdp annual growth rate,gdp growth rate yoy prel,FRGEGDPY,,0.6%,0.2%
612,2024-01-30,italy,gdp growth rate,gdp growth rate qoq adv,ITPIRLQS,,0.1%,0.2%
613,2024-01-30,italy,gdp annual growth rate,gdp growth rate yoy adv,ITPIRLYS,,0.1%,-0.3%
614,2024-01-30,euro area,gdp growth rate,gdp growth rate qoq flash,EUGNEMUQ,,-0.1%,-0.1%


In [37]:
calendar_data_df_jp = calendar_data_df[calendar_data_df.country == "japan"].copy()

In [38]:
calendar_data_df_jp.head(10)

,date,country,category,event,symbol,actual,previous,forecast
1,2024-01-23,japan,interest rate,boj interest rate decision,BOJDTR,,-0.1%,-0.1%
2,2024-01-23,japan,balance of trade,balance of trade,JNTBAL,,¥-776.9B,¥ -200B
23,2024-01-23,japan,interest rate,boj interest rate decision,BOJDTR,,-0.1%,-0.1%
24,2024-01-23,japan,balance of trade,balance of trade,JNTBAL,,¥-776.9B,¥ -200B
45,2024-01-23,japan,interest rate,boj interest rate decision,BOJDTR,,-0.1%,-0.1%
46,2024-01-23,japan,balance of trade,balance of trade,JNTBAL,,¥-776.9B,¥ -200B
67,2024-01-23,japan,interest rate,boj interest rate decision,BOJDTR,,-0.1%,-0.1%
68,2024-01-23,japan,balance of trade,balance of trade,JNTBAL,,¥-776.9B,¥ -200B
89,2024-01-23,japan,interest rate,boj interest rate decision,BOJDTR,,-0.1%,-0.1%
90,2024-01-23,japan,balance of trade,balance of trade,JNTBAL,,¥-776.9B,¥ -200B


In [41]:
calendar_data_df_jp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 56 entries, 1 to 596
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   date      56 non-null     object 
 1   country   56 non-null     object 
 2   category  56 non-null     object 
 3   event     56 non-null     object 
 4   symbol    56 non-null     object 
 5   actual    56 non-null     float64
 6   previous  56 non-null     float64
 7   forecast  56 non-null     float64
dtypes: float64(3), object(5)
memory usage: 3.9+ KB


In [40]:
for col in ['actual', 'previous', 'forecast']:
    for sy in ['¥', '%', 'B', 'K']:
        calendar_data_df_jp[col] = calendar_data_df_jp[col].str.replace(sy, "")
    calendar_data_df_jp[col].replace('', 0, inplace=True)
    calendar_data_df_jp[col] = calendar_data_df_jp[col].astype(float)

In [42]:
date_match= {}
for orig in calendar_data_df_jp.date.unique():
    d = orig
    tries = 0
    while d not in candles_df.time.values:
        d = d + dt.timedelta(days=1)
        tries += 1
        if tries > 5:
            print("Failed")
            break
    date_match[orig] = d

Failed


In [43]:
date_match

{datetime.date(2024, 1, 23): datetime.date(2024, 1, 29)}

In [44]:
calendar_data_df_jp['orig_date'] = calendar_data_df_jp.date
calendar_data_df_jp.date = [date_match[x] for x in calendar_data_df_jp.date]  

In [45]:
calendar_data_df_jp['delta_prev'] = calendar_data_df_jp.actual - calendar_data_df_jp.previous
calendar_data_df_jp['delta_fc'] = calendar_data_df_jp.actual - calendar_data_df_jp.forecast

In [46]:
calendar_data_df_jp.head(2)

,date,country,category,event,symbol,actual,previous,forecast,orig_date,delta_prev,delta_fc
1,2024-01-29,japan,interest rate,boj interest rate decision,BOJDTR,0.0,-0.1,-0.1,2024-01-23,0.1,0.1
2,2024-01-29,japan,balance of trade,balance of trade,JNTBAL,0.0,-776.9,-200.0,2024-01-23,776.9,200.0


In [47]:
candles_df.head(2)

,volume,time,mid-o,mid-h,mid-l,mid-c,bid-o,bid-h,bid-l,bid-c,ask-o,ask-h,ask-l,ask-c,gain,pair
0,91374,2022-07-06,0.97031,0.97487,0.96818,0.97412,0.96956,0.97480,0.96810,0.97396,0.97106,0.97496,0.96826,0.97428,0.391122,USD_CHF
1,94364,2022-07-07,0.97342,0.97978,0.97232,0.97666,0.97267,0.97968,0.97186,0.97625,0.97417,0.97989,0.97239,0.97708,0.331743,USD_CHF


In [48]:
candles_an = candles_df[['time', 'pair', 'gain']].copy()

In [49]:
candles_an.head(2)

,time,pair,gain
0,2022-07-06,USD_CHF,0.391122
1,2022-07-07,USD_CHF,0.331743


In [50]:
merged = pd.merge(left=candles_an, right=calendar_data_df_jp, left_on='time', right_on='date')

In [51]:
merged.head()

,time,pair,gain,date,country,category,event,symbol,actual,previous,forecast,orig_date,delta_prev,delta_fc
